In [15]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.options.mode.chained_assignment = None

In [4]:
test = pd.read_csv('../data/test.csv', usecols=['item_id'])

In [6]:
periods = pd.read_csv('periods.csv')

In [ ]:
tabular = pd.read_csv('tabular.csv')

In [ ]:
nlp = pd.read_csv('nlp.csv')

In [7]:
test['periods_weight'] = 1.0
test['tabular_weight'] = 1.0
test['nlp_weight'] = 1.0

In [16]:
len(test)

508438

In [17]:
test.isna().sum() / len(test)

item_id           0.0
periods_weight    0.0
tabular_weight    0.0
nlp_weight        0.0
periods           0.0
dtype: float64

In [14]:
test = test.merge(periods, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'periods'})
test['periods_weight'][null_idx] = 0.0
test['periods'][null_idx] = 0.0

/Users/alirezafaghaninia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/alirezafaghaninia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
test = test.merge(tabular, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'tabular'})
test['tabular_weight'][null_idx] = 0.0
test['tabular'][null_idx] = 0.0

In [ ]:
test = test.merge(nlp, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'nlp'})
test['nlp_weight'][null_idx] = 0.0
test['nlp'][null_idx] = 0.0

In [ ]:
test['deal_probability'] = test['periods']*test['periods_weight'] + \
                            test['tabular']*test['tabular_weight'] + \
                            test['nlp']*test['nlp_weight']
test['deal_probability'] /= ( test['periods_weight'] + test['tabular_weight'] + test['nlp_weight'])

In [ ]:
test[['item_id', 'deal_probability']].to_csv('submission.csv')